<a id='import'></a>
## 1. Import Libraries

In [1]:
# Data manipulation
import pandas as pd
import numpy as np

# Text processing
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Custom modules
import sys
sys.path.append('../src')
from data_preprocessing import create_text_features

# Utilities
import warnings
import os
import joblib

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

print("Libraries imported successfully!")

Libraries imported successfully!


<a id='load'></a>
## 2. Load Cleaned Data

In [2]:
# Load preprocessed data
train_df = pd.read_csv('../data/processed/train_cleaned.csv')
test_df = pd.read_csv('../data/processed/test_cleaned.csv')

print("Data loaded:")
print(f"Training: {train_df.shape}")
print(f"Test: {test_df.shape}")
print("\n" + "="*80)

# Separate features and target
y_train = train_df['fraudulent']
X_train = train_df.drop('fraudulent', axis=1)
X_test = test_df.copy()

print(f"\nTarget distribution:")
print(y_train.value_counts())
print(f"\nFraudulent percentage: {(y_train.sum() / len(y_train)) * 100:.2f}%")

Data loaded:
Training: (9999, 18)
Test: (7881, 17)


Target distribution:
fraudulent
0    9514
1     485
Name: count, dtype: int64

Fraudulent percentage: 4.85%


<a id='text-features'></a>
## 3. Basic Text Features

Create length-based and statistical features from text columns

In [3]:
# Create text features for train and test
print("Creating text features...\n")

X_train = create_text_features(X_train)
X_test = create_text_features(X_test)

print("\nText features created!")
print(f"\nNew shape - Train: {X_train.shape}, Test: {X_test.shape}")

Creating text features...


Text features created!

New shape - Train: (9999, 32), Test: (7881, 32)


In [4]:
# Display newly created features
text_feature_cols = [col for col in X_train.columns if any(x in col for x in ['_length', '_word_count', '_has_', 'caps_ratio', 'special_char'])]

print(f"Created {len(text_feature_cols)} text-based features:")
print("="*80)
for col in text_feature_cols[:20]:  # Show first 20
    print(f"  - {col}")
if len(text_feature_cols) > 20:
    print(f"  ... and {len(text_feature_cols) - 20} more")

Created 9 text-based features:
  - title_length
  - description_length
  - requirements_length
  - benefits_length
  - company_profile_length
  - title_word_count
  - description_word_count
  - requirements_word_count
  - description_special_chars


<a id='tfidf'></a>
## 4. TF-IDF Features

Create TF-IDF features from key text columns

In [5]:
# Combine important text fields
def combine_text_fields(df):
    """Combine key text fields into one column"""
    text_fields = ['title', 'company_profile', 'description', 'requirements']
    combined = df[text_fields].fillna('').astype(str).agg(' '.join, axis=1)
    return combined

# Create combined text
train_text = combine_text_fields(X_train)
test_text = combine_text_fields(X_test)

print("Combined text fields for TF-IDF")
print(f"Sample: {train_text.iloc[0][:200]}...")

Combined text fields for TF-IDF
Sample: Office Manager / HR  Based in the heart of the City, this exceptional opportunity as Office Manager / HR in a fast-growing tech start-up is a challenging and varied position requiring a highly motivat...


In [6]:
# Create TF-IDF features
print("Creating TF-IDF features...\n")

# Initialize TF-IDF vectorizer
tfidf = TfidfVectorizer(
    max_features=500,  # Top 500 features
    min_df=5,          # Minimum document frequency
    max_df=0.8,        # Maximum document frequency
    ngram_range=(1, 2), # Unigrams and bigrams
    stop_words='english'
)

# Fit on training data and transform both
tfidf_train = tfidf.fit_transform(train_text)
tfidf_test = tfidf.transform(test_text)

print(f"TF-IDF features created!")
print(f"Shape: {tfidf_train.shape}")
print(f"Feature names (first 10): {tfidf.get_feature_names_out()[:10]}")

Creating TF-IDF features...

TF-IDF features created!
Shape: (9999, 500)
Feature names (first 10): ['000' '10' '100' '30' 'ability' 'ability work' 'able' 'abroad' 'access'
 'account']


In [7]:
# Convert TF-IDF to DataFrame
tfidf_train_df = pd.DataFrame(
    tfidf_train.toarray(), 
    columns=[f'tfidf_{name}' for name in tfidf.get_feature_names_out()],
    index=X_train.index
)

tfidf_test_df = pd.DataFrame(
    tfidf_test.toarray(), 
    columns=[f'tfidf_{name}' for name in tfidf.get_feature_names_out()],
    index=X_test.index
)

print("TF-IDF DataFrames created")
print(f"Train shape: {tfidf_train_df.shape}")
print(f"Test shape: {tfidf_test_df.shape}")

TF-IDF DataFrames created
Train shape: (9999, 500)
Test shape: (7881, 500)


<a id='categorical'></a>
## 5. Categorical Encoding

In [8]:
# Define categorical columns
categorical_cols = ['employment_type', 'required_experience', 'required_education', 'industry', 'function']
binary_cols = ['telecommuting', 'has_company_logo', 'has_questions']

print("Categorical columns to encode:")
for col in categorical_cols:
    unique_count = X_train[col].nunique()
    print(f"  - {col}: {unique_count} unique values")

Categorical columns to encode:
  - employment_type: 6 unique values
  - required_experience: 8 unique values
  - required_education: 14 unique values
  - industry: 126 unique values
  - function: 38 unique values


In [9]:
# Label encoding for categorical variables
print("\nEncoding categorical variables...\n")

label_encoders = {}

for col in categorical_cols:
    le = LabelEncoder()
    
    # Fit on combined train+test to ensure consistent encoding
    combined = pd.concat([X_train[col], X_test[col]])
    le.fit(combined)
    
    # Transform
    X_train[f'{col}_encoded'] = le.transform(X_train[col])
    X_test[f'{col}_encoded'] = le.transform(X_test[col])
    
    label_encoders[col] = le
    print(f"Encoded: {col}")

print("\nCategorical encoding completed!")


Encoding categorical variables...

Encoded: employment_type
Encoded: required_experience
Encoded: required_education
Encoded: industry
Encoded: function

Categorical encoding completed!


In [10]:
# Select features for modeling
# Keep: binary features, encoded categorical, text features
# Drop: original categorical and text columns, job_id

columns_to_drop = ['job_id'] + categorical_cols + ['title', 'location', 'department', 'salary_range', 
                                                     'company_profile', 'description', 'requirements', 'benefits']

X_train_features = X_train.drop(columns=columns_to_drop, errors='ignore')
X_test_features = X_test.drop(columns=columns_to_drop, errors='ignore')

print(f"Feature set shape (before TF-IDF):")
print(f"Train: {X_train_features.shape}")
print(f"Test: {X_test_features.shape}")

Feature set shape (before TF-IDF):
Train: (9999, 23)
Test: (7881, 23)


<a id='selection'></a>
## 6. Combine All Features

In [11]:
# Combine basic features with TF-IDF features
X_train_final = pd.concat([X_train_features.reset_index(drop=True), tfidf_train_df.reset_index(drop=True)], axis=1)
X_test_final = pd.concat([X_test_features.reset_index(drop=True), tfidf_test_df.reset_index(drop=True)], axis=1)

print("Final Feature Set:")
print("="*80)
print(f"Train shape: {X_train_final.shape}")
print(f"Test shape: {X_test_final.shape}")
print(f"Total features: {X_train_final.shape[1]}")
print(f"\nFeature categories:")
print(f"  - Binary features: {len(binary_cols)}")
print(f"  - Encoded categorical: {len(categorical_cols)}")
print(f"  - Text features: {len(text_feature_cols)}")
print(f"  - TF-IDF features: {tfidf_train_df.shape[1]}")

Final Feature Set:
Train shape: (9999, 523)
Test shape: (7881, 523)
Total features: 523

Feature categories:
  - Binary features: 3
  - Encoded categorical: 5
  - Text features: 9
  - TF-IDF features: 500


In [12]:
# Display sample of final features
print("\nSample of final feature set:")
X_train_final.head()


Sample of final feature set:


,telecommuting,has_company_logo,has_questions,title_length,description_length,requirements_length,benefits_length,company_profile_length,title_word_count,description_word_count,requirements_word_count,has_salary,has_department,has_company_profile,has_requirements,has_benefits,description_special_chars,title_capital_ratio,employment_type_encoded,required_experience_encoded,required_education_encoded,industry_encoded,function_encoded,tfidf_000,tfidf_10,tfidf_100,tfidf_30,tfidf_ability,tfidf_ability work,tfidf_able,tfidf_abroad,tfidf_access,tfidf_account,tfidf_accounting,tfidf_accounts,tfidf_achieve,tfidf_activities,tfidf_administrative,tfidf_advertising,tfidf_agency,tfidf_agile,tfidf_amp,tfidf_analysis,tfidf_analytical,tfidf_analytics,tfidf_app,tfidf_application,tfidf_applications,tfidf_apply,tfidf_approach,tfidf_appropriate,tfidf_apps,tfidf_area,tfidf_areas,tfidf_assigned,tfidf_assist,tfidf_assistant,tfidf_attention,tfidf_attitude,tfidf_available,tfidf_awesome,tfidf_background,tfidf_based,tfidf_basic,tfidf_believe,tfidf_benefits,tfidf_best,tfidf_better,tfidf_big,tfidf_brand,tfidf_brands,tfidf_bring,tfidf_build,tfidf_building,tfidf_business,tfidf_businesses,tfidf_calls,tfidf_campaigns,tfidf_candidate,tfidf_candidates,tfidf_capital,tfidf_care,tfidf_career,tfidf_center,tfidf_challenges,tfidf_change,tfidf_changing,tfidf_city,tfidf_class,tfidf_client,tfidf_clients,tfidf_closely,tfidf_cloud,tfidf_code,tfidf_come,tfidf_commercial,tfidf_committed,tfidf_communicate,tfidf_communication,tfidf_communication skills,tfidf_communications,tfidf_community,tfidf_companies,tfidf_company,tfidf_competitive,tfidf_complete,tfidf_complex,tfidf_compliance,tfidf_computer,tfidf_consulting,tfidf_consumer,tfidf_consumers,tfidf_contact,tfidf_content,tfidf_contract,tfidf_control,tfidf_core,tfidf_corporate,tfidf_cost,tfidf_create,tfidf_creating,tfidf_creative,tfidf_credit,tfidf_cross,tfidf_culture,tfidf_current,tfidf_currently,tfidf_customer,tfidf_customer service,tfidf_customers,tfidf_daily,tfidf_data,tfidf_database,tfidf_day,tfidf_degree,tfidf_deliver,tfidf_delivering,tfidf_delivery,tfidf_department,tfidf_design,tfidf_designer,tfidf_designers,tfidf_develop,tfidf_developer,tfidf_developers,tfidf_developing,tfidf_development,tfidf_different,tfidf_digital,tfidf_direct,tfidf_directly,tfidf_director,tfidf_document,tfidf_documentation,tfidf_don,tfidf_drive,tfidf_driven,tfidf_driving,tfidf_duties,tfidf_dynamic,tfidf_education,tfidf_effective,tfidf_effectively,tfidf_email,tfidf_employee,tfidf_employees,tfidf_employer,tfidf_employment,tfidf_end,tfidf_engineer,tfidf_engineering,tfidf_engineers,tfidf_english,tfidf_ensure,tfidf_ensuring,tfidf_enterprise,tfidf_entry,tfidf_environment,tfidf_equipment,tfidf_equivalent,tfidf_essential,tfidf_established,tfidf_events,tfidf_excel,tfidf_excellent,tfidf_exceptional,tfidf_exciting,tfidf_executive,tfidf_existing,tfidf_experience working,tfidf_experienced,tfidf_experiences,tfidf_expert,tfidf_expertise,tfidf_family,tfidf_fast,tfidf_fast paced,tfidf_field,tfidf_finance,tfidf_financial,tfidf_firm,tfidf_flexible,tfidf_focus,tfidf_focused,tfidf_follow,tfidf_following,tfidf_food,tfidf_founded,tfidf_free,tfidf_fun,tfidf_functional,tfidf_functions,tfidf_future,tfidf_general,tfidf_global,tfidf_goal,tfidf_goals,tfidf_good,tfidf_great,tfidf_group,tfidf_grow,tfidf_growing,tfidf_growth,tfidf_hands,tfidf_hard,tfidf_health,tfidf_help,tfidf_helping,tfidf_high,tfidf_high quality,tfidf_highest,tfidf_highly,tfidf_hiring,tfidf_home,tfidf_hours,tfidf_house,tfidf_hr,tfidf_human,tfidf_ideal,tfidf_ideas,tfidf_identify,tfidf_impact,tfidf_implement,tfidf_implementation,tfidf_improve,tfidf_include,tfidf_including,tfidf_increase,tfidf_independently,tfidf_individual,tfidf_individuals,tfidf_industry,tfidf_information,tfidf_innovation,tfidf_innovative,tfidf_insurance,tfidf_interested,tfidf_internal,tfidf_international,tfidf_internet,tfidf_interpersonal,tfidf_issues,tfidf_java,tfidf_javascript,tfidf_job,tfidf_jobs,tfidf_join,tfidf_just,tfidf_key,tfidf_know,tfidf

In [13]:
# Check for any missing values
print("Data quality check:")
print("="*80)
print(f"Missing values in train: {X_train_final.isnull().sum().sum()}")
print(f"Missing values in test: {X_test_final.isnull().sum().sum()}")
print(f"Infinite values in train: {np.isinf(X_train_final.select_dtypes(include=[np.number])).sum().sum()}")
print(f"Infinite values in test: {np.isinf(X_test_final.select_dtypes(include=[np.number])).sum().sum()}")

Data quality check:
Missing values in train: 0
Missing values in test: 0
Infinite values in train: 0
Infinite values in test: 0


<a id='save'></a>
## 7. Save Feature Sets

In [14]:
# Create directory for processed features
os.makedirs('../data/features', exist_ok=True)

# Save feature sets
X_train_final.to_csv('../data/features/X_train.csv', index=False)
X_test_final.to_csv('../data/features/X_test.csv', index=False)
y_train.to_csv('../data/features/y_train.csv', index=False, header=True)

print("Feature sets saved!")
print("\nSaved files:")
print("  - ../data/features/X_train.csv")
print("  - ../data/features/X_test.csv")
print("  - ../data/features/y_train.csv")

Feature sets saved!

Saved files:
  - ../data/features/X_train.csv
  - ../data/features/X_test.csv
  - ../data/features/y_train.csv


In [15]:
# Save encoders and vectorizers for later use
os.makedirs('../models/encoders', exist_ok=True)

joblib.dump(tfidf, '../models/encoders/tfidf_vectorizer.pkl')
joblib.dump(label_encoders, '../models/encoders/label_encoders.pkl')

print("Encoders and vectorizers saved!")
print("\nSaved files:")
print("  - ../models/encoders/tfidf_vectorizer.pkl")
print("  - ../models/encoders/label_encoders.pkl")

Encoders and vectorizers saved!

Saved files:
  - ../models/encoders/tfidf_vectorizer.pkl
  - ../models/encoders/label_encoders.pkl


---

## Summary

### Completed Tasks:
✅ Loaded cleaned datasets
✅ Created text-based features (length, word counts, special characters)
✅ Generated TF-IDF features from combined text
✅ Encoded categorical variables
✅ Combined all features into final dataset
✅ Saved feature sets and encoders

### Feature Summary:
- **Total Features**: ~500+ features
- **Binary**: 3 features (telecommuting, has_company_logo, has_questions)
- **Categorical**: 5 encoded features
- **Text Statistics**: ~15 features (lengths, word counts, etc.)
- **TF-IDF**: 500 features from text content

### Next Steps:
- Model training (Notebook 04)
- Handle class imbalance (SMOTE, class weights)
- Train multiple models (Logistic Regression, Random Forest, XGBoost)
- Hyperparameter tuning
- Model evaluation and selection